# Image Feature Extraction

In [ ]:
import numpy as np

# skimage feature + morphology + measurement
from skimage.feature import local_binary_pattern, hog as sk_hog
from skimage.morphology import skeletonize
from skimage.measure import label, regionprops, moments_hu, euler_number

# numeric helpers
from scipy.ndimage import convolve

# ML utils
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pickle

In [ ]:
train_path = "output/emnist_train_processed.npz"
test_path = "output/emnist_test_processed.npz"

In [ ]:
with np.load(train_path) as data:
    X_train = data['X_train']
    y_train = data['y_train']

with np.load(test_path) as data:
    X_test = data['X_test']
    y_test = data['y_test']

print(f"Train set shape: {X_train.shape}\tTrain labels shape: {y_train.shape}")
print(f"Test set shape: {X_test.shape}\tTest labels shape: {y_test.shape}")

In [ ]:
def zoning(img):
    if img is None:
        return np.zeros(72, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    img = img.astype(np.float32, copy=False)
    bin_img = (img > 0).astype(np.float32)
    h, w = bin_img.shape
    gh, gw = h // 4, w // 4
    densities = []
    for i in range(4):
        for j in range(4):
            patch = bin_img[i*gh:(i+1)*gh, j*gw:(j+1)*gw]
            densities.append(patch.mean())
    proj_h = bin_img.sum(axis=1)
    proj_v = bin_img.sum(axis=0)
    proj_h = proj_h / (proj_h.sum() + 1e-7)
    proj_v = proj_v / (proj_v.sum() + 1e-7)
    return np.concatenate([np.array(densities, dtype=np.float32),
                           proj_h.astype(np.float32),
                           proj_v.astype(np.float32)], dtype=np.float32, copy=False)


def hog(img):
    if img is None:
        return np.zeros(1, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    img = img.astype(np.float32, copy=False)
    if img.max() > 1.0:
        img = img / 255.0
    features = sk_hog(img,
                      orientations=9,
                      pixels_per_cell=(4, 4),
                      cells_per_block=(2, 2),
                      block_norm='L2-Hys',
                      transform_sqrt=True,
                      visualize=False,
                      feature_vector=True)
    return features.astype(np.float32, copy=False)


def hu(img):
    if img is None:
        return np.zeros(7, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.float32)
    vals = moments_hu(bin_img)
    vals = -np.sign(vals) * np.log10(np.abs(vals) + 1e-10)
    return vals.astype(np.float32, copy=False)


def lbp(img):
    if img is None:
        return np.zeros(10, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    img = img.astype(np.float32, copy=False)
    lbp_img = local_binary_pattern(img, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp_img.ravel(), bins=10, range=(0, 10))
    hist = hist.astype(np.float32)
    hist /= (hist.sum() + 1e-7)
    return hist.astype(np.float32, copy=False)


def skeleton(img):
    if img is None:
        return np.zeros(4, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.uint8)
    skel = skeletonize(bin_img > 0)
    skel_uint = skel.astype(np.uint8)
    kernel = np.array([[1,1,1],[1,0,1],[1,1,1]], dtype=np.uint8)
    neighbor_count = convolve(skel_uint, kernel, mode='constant', cval=0)
    endpoints = np.sum((skel_uint == 1) & (neighbor_count == 1))
    branches = np.sum((skel_uint == 1) & (neighbor_count >= 3))
    length = skel_uint.sum()
    area = bin_img.sum()
    ratio = length / (area + 1e-7)
    return np.array([length, endpoints, branches, ratio], dtype=np.float32, copy=False)


def countours(img):
    if img is None:
        return np.zeros(6, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.uint8)
    lbl = label(bin_img)
    props = regionprops(lbl)
    if not props:
        return np.zeros(6, dtype=np.float32)
    main = max(props, key=lambda r: r.area)
    minr, minc, maxr, maxc = main.bbox
    height = maxr - minr
    width = maxc - minc
    aspect = width / (height + 1e-7)
    perimeter = main.perimeter if hasattr(main, 'perimeter') else 0.0
    feats = [main.area, main.eccentricity, main.extent, main.solidity, aspect, perimeter]
    return np.array(feats, dtype=np.float32, copy=False)


def pixel_run_length(img):
    if img is None:
        return np.zeros(6, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.uint8)

    def runs_1d(arr):
        runs = []
        count = 0
        for v in arr:
            if v == 1:
                count += 1
            elif count > 0:
                runs.append(count)
                count = 0
        if count > 0:
            runs.append(count)
        if not runs:
            return [0]
        return runs

    horiz_runs = []
    for row in bin_img:
        horiz_runs.extend(runs_1d(row))
    vert_runs = []
    for col in bin_img.T:
        vert_runs.extend(runs_1d(col))

    horiz_runs = np.array(horiz_runs, dtype=np.float32)
    vert_runs = np.array(vert_runs, dtype=np.float32)

    def stats(vec):
        return np.array([vec.mean(), vec.std(), vec.max()], dtype=np.float32) if vec.size else np.zeros(3, dtype=np.float32)

    return np.concatenate([stats(horiz_runs), stats(vert_runs)], dtype=np.float32, copy=False)


def connectivity(img):
    """Devuelve: [n_components, euler_number]"""
    if img is None:
        return np.zeros(2, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.uint8)
    lbl = label(bin_img)
    n_components = int(lbl.max())
    try:
        euler = float(euler_number(bin_img))
    except Exception:
        # fallback: sum of region euler numbers
        props = regionprops(lbl)
        euler = float(sum([p.euler_number for p in props])) if props else 0.0
    return np.array([n_components, euler], dtype=np.float32, copy=False)


def geometry(img):
    """Devuelve 7 features: bbox(4), centroid(2), moment_of_inertia(1)"""
    if img is None:
        return np.zeros(7, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.uint8)
    lbl = label(bin_img)
    props = regionprops(lbl)
    if not props:
        return np.zeros(7, dtype=np.float32)
    main = max(props, key=lambda r: r.area)
    minr, minc, maxr, maxc = main.bbox
    cy, cx = main.centroid
    # momento de inercia: sum((x-cx)^2 + (y-cy)^2) sobre pixeles foreground
    coords = np.column_stack(np.nonzero(bin_img))
    if coords.size == 0:
        inertia = 0.0
    else:
        dy = coords[:,0] - cy
        dx = coords[:,1] - cx
        inertia = float(np.sum(dx*dx + dy*dy)) / (coords.shape[0] + 1e-7)
    return np.array([minr, minc, maxr, maxc, cy, cx, inertia], dtype=np.float32, copy=False)


def projections(img):
    """24 features: horizontal hist (10), vertical hist (10), diagonals (4)"""
    if img is None:
        return np.zeros(24, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.float32)
    n = bin_img.shape[0]
    # horizontal and vertical projections (sums)
    proj_h = bin_img.sum(axis=1)
    proj_v = bin_img.sum(axis=0)
    # histogram into 10 bins each (bins computed over possible range 0..n)
    h_hist, _ = np.histogram(proj_h, bins=10, range=(0, n))
    v_hist, _ = np.histogram(proj_v, bins=10, range=(0, n))
    h_hist = h_hist.astype(np.float32) / (h_hist.sum() + 1e-7)
    v_hist = v_hist.astype(np.float32) / (v_hist.sum() + 1e-7)
    # diagonals: sum over all diagonals then group into 4 groups
    diag_sums = []
    for k in range(-n+1, n):
        diag_sums.append(np.sum(np.diag(bin_img, k=k)))
    diag_sums = np.array(diag_sums, dtype=np.float32)
    # group into 4 contiguous bins
    m = len(diag_sums)
    group_size = m // 4
    diag_feats = []
    for i in range(4):
        start = i*group_size
        end = (i+1)*group_size if i < 3 else m
        s = diag_sums[start:end].sum()
        diag_feats.append(s)
    diag_feats = np.array(diag_feats, dtype=np.float32)
    if diag_feats.sum() > 0:
        diag_feats = diag_feats / (diag_feats.sum() + 1e-7)
    return np.concatenate([h_hist, v_hist, diag_feats], dtype=np.float32, copy=False)


def density_quadrants(img):
    """Devuelve 2 features: densidad izquierda, densidad derecha (suma de dos cuadrantes cada una)."""
    if img is None:
        return np.zeros(2, dtype=np.float32)
    if img.ndim == 1:
        side = int(np.sqrt(img.size))
        img = img.reshape(side, side)
    bin_img = (img > 0).astype(np.float32)
    h, w = bin_img.shape
    mid_h = h // 2
    mid_w = w // 2
    tl = bin_img[:mid_h, :mid_w].sum()
    tr = bin_img[:mid_h, mid_w:].sum()
    bl = bin_img[mid_h:, :mid_w].sum()
    br = bin_img[mid_h:, mid_w:].sum()
    left = (tl + bl) / (bin_img.sum() + 1e-7)
    right = (tr + br) / (bin_img.sum() + 1e-7)
    return np.array([left, right], dtype=np.float32, copy=False)


def extract_features_img(img):
    fz = zoning(img)
    fh = hog(img)
    fhu = hu(img)
    fl = lbp(img)
    fs = skeleton(img)
    fc = countours(img)
    fr = pixel_run_length(img)
    fconn = connectivity(img)
    fgeo = geometry(img)
    fproj = projections(img)
    fdens = density_quadrants(img)
    feats = np.concatenate([fz, fh, fhu, fl, fs, fc, fr, fconn, fgeo, fproj, fdens])
    return feats.astype(np.float32, copy=False)


def extract_features(X):
    n_samples = X.shape[0]
    feature_list = []
    for i in range(n_samples):
        img = X[i]
        feats = extract_features_img(img)
        feature_list.append(feats)
    return np.array(feature_list, dtype=np.float32)

In [ ]:
train_features = extract_features(X_train)
test_features = extract_features(X_test)